In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
# Download the data, if not already on disk and load it as numpy arrays
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

In [3]:
# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape
# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]
# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]
y = tf.one_hot(y, depth=n_classes) ####
print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


In [4]:
y.shape

TensorShape([1288, 7])

In [5]:
#normalise data
X = X/255
X

array([[0.99607843, 0.99607843, 0.9869281 , ..., 0.34248367, 0.3477124 ,
        0.33986926],
       [0.15555556, 0.19738561, 0.18431373, ..., 0.4614379 , 0.4509804 ,
        0.52418303],
       [0.3503268 , 0.40784314, 0.49411765, ..., 0.68758166, 0.7189542 ,
        0.7176471 ],
       ...,
       [0.3372549 , 0.3150327 , 0.29281044, ..., 0.17254902, 0.19477125,
        0.1751634 ],
       [0.19738561, 0.25751632, 0.34509805, ..., 0.77254903, 0.70326793,
        0.65228754],
       [0.11764706, 0.10588235, 0.12810458, ..., 0.13725491, 0.13856208,
        0.23921569]], dtype=float32)

In [6]:
#The model expects an input with 4 dimensions
#reshape data with size (n_images, rows, cols, channels)
X = X.reshape(n_samples, h, w, 1)

In [7]:
X.shape

(1288, 50, 37, 1)

In [8]:
# Split into a training set and a test set using a stratified k fold
X_train, X_test, y_train, y_test = train_test_split(X, y.numpy(), test_size=0.25, random_state=42)

In [9]:
y_train.shape

(966, 7)

In [8]:
X_train.shape

(966, 50, 37, 1)

In [9]:
from tensorflow import keras

In [10]:
model = tf.keras.Sequential()

#Add two convolution layers of 3x3 with 32 filters each
#input shape = (rows, cols, channels) 
#image size 50 *37, #no of channels = 1(grayscale)
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation='relu', input_shape=(h,w,1)))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))

#Adding Dense layers for classification
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(7, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 35, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 17, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 15, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 7, 32)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2464)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1262080   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1

In [11]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['acc'])

In [12]:
history = model.fit(X_train, y_train, epochs=20)

Epoch 1/20
31/31 [==============================] - 5s 7ms/step - loss: 1.7447 - acc: 0.3851
Epoch 2/20
31/31 [==============================] - 0s 5ms/step - loss: 1.6875 - acc: 0.3975
Epoch 3/20
31/31 [==============================] - 0s 5ms/step - loss: 1.5810 - acc: 0.4296
Epoch 4/20
31/31 [==============================] - 0s 4ms/step - loss: 1.2666 - acc: 0.5424
Epoch 5/20
31/31 [==============================] - 0s 4ms/step - loss: 0.8974 - acc: 0.6936
Epoch 6/20
31/31 [==============================] - 0s 4ms/step - loss: 0.7045 - acc: 0.7567
Epoch 7/20
31/31 [==============================] - 0s 4ms/step - loss: 0.5607 - acc: 0.8116
Epoch 8/20
31/31 [==============================] - 0s 4ms/step - loss: 0.4041 - acc: 0.8654
Epoch 9/20
31/31 [==============================] - 0s 4ms/step - loss: 0.3340 - acc: 0.8892
Epoch 10/20
31/31 [==============================] - 0s 4ms/step - loss: 0.2798 - acc: 0.9048
Epoch 11/20
31/31 [==============================] - 0s 4ms/step - lo

In [13]:
#Testing
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test_acc:", test_acc)

11/11 [==============================] - 0s 5ms/step - loss: 0.7587 - acc: 0.8789
Test_acc: 0.8788819909095764
